In [1]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from mne import io, read_events, pick_types, Epochs
from mne.datasets import sample
from mne.preprocessing import ICA, create_eog_epochs
import mne

In [4]:
# Set up parameters
random_state = 23
n_components = 25  # For ICA
l_freq = 1.0  # High-pass filter cutoff
h_freq = 45.0  # Low-pass filter cutoff

# Sample data path
data_path = sample.data_path()
raw_fname = data_path / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
event_fname = data_path / 'MEG' / 'sample' / 'sample_audvis_raw-eve.fif'

# Load the raw data
raw = io.read_raw_fif(raw_fname, preload=True)

# Keep only EEG and EOG channels
raw.pick_types(meg=False, eeg=True, eog=True, stim=True, exclude='bads')

# Filter the data
raw.filter(l_freq=l_freq, h_freq=h_freq)

# Plot raw data
raw.plot(duration=5, n_channels=20, title='Raw EEG and EOG signals')
plt.show()


Opening raw data file /home/koutras/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1983 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass


Channels marked as bad:
none


/opt/anaconda3/envs/biosignals/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py:3061: RuntimeWarning: Failed to disconnect (None) from signal "triggered()".
  sig.disconnect()


In [5]:
# Load events
events = read_events(event_fname)

# Create epochs
tmin, tmax = -0.2, 0.5
event_id = {'Auditory/Left': 1, 'Auditory/Right': 2}
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=False,
               picks=['eeg', 'eog'], baseline=(None, 0), preload=True)

# Plot epochs before artifact rejection
epochs.plot(n_epochs=5, title='Epochs before artifact rejection')

# Helper function to visualize EOG artifacts
def plot_eog_artifacts(raw_data, eog_evoked, title):
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    plt.plot(eog_evoked.times, eog_evoked.data.T)
    plt.title(f'{title} - EOG Evoked Response')
    plt.xlabel('Time (s)')
    plt.ylabel('uV')
    
    plt.subplot(2, 1, 2)
    plt.plot(raw_data.times[:1000], raw_data.get_data(picks='eog')[:, :1000].T)
    plt.title(f'{title} - Sample EOG Signal')
    plt.xlabel('Time (s)')
    plt.ylabel('uV')
    plt.tight_layout()
    plt.show()

# Create EOG epochs for visualization
eog_epochs = create_eog_epochs(raw, ch_name='EOG 061')
eog_evoked = eog_epochs.average()

# Plot EOG artifacts
plot_eog_artifacts(raw, eog_evoked, 'Before Rejection')

# Make a copy of raw and epochs for comparing methods
raw_ssp = raw.copy()
epochs_ssp = epochs.copy()



Not setting metadata
145 matching events found
Setting baseline interval to [-0.19979521315838786, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 145 events and 421 original time points ...


0 bad epochs dropped


qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass


Using EOG channel: EOG 061
EOG channel index for this subject is: [68]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG 061 for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 6007 samples (10.001 s)

Now detecting blinks and generating corresponding events
Found 46 significant peaks
Number of EOG events detected: 46
Not setting metadata
46 matching events found
No baseline correction applied
Using data from preloaded Raw for 46 events and 601 original time points ...
0 bad epochs dropped


Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


/opt/anaconda3/envs/biosignals/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py:3061: RuntimeWarning: Failed to disconnect (None) from signal "triggered()".
  sig.disconnect()


In [6]:
######################################
# Method 1: ICA for artifact rejection
#######################################

# Initialize ICA
ica = ICA(n_components=n_components, random_state=random_state, method='fastica')

# Fit ICA on filtered raw data
ica.fit(raw)

# Plot ICA components
ica.plot_components()

# Find EOG artifacts using correlation with EOG channel
eog_indices, eog_scores = ica.find_bads_eog(raw)
ica.plot_scores(eog_scores)

# Plot properties of detected EOG components
if eog_indices:
    ica.plot_properties(raw, picks=eog_indices)

# Exclude identified components and apply ICA correction
ica.exclude = eog_indices
reconst_raw = raw.copy()
ica.apply(reconst_raw)

# Create epochs after ICA correction
epochs_ica = Epochs(reconst_raw, events, event_id, tmin, tmax, proj=False,
                   picks=['eeg', 'eog'], baseline=(None, 0), preload=True)

# Create EOG epochs after ICA for visualization
eog_epochs_ica = create_eog_epochs(reconst_raw, ch_name='EOG 061')
eog_evoked_ica = eog_epochs_ica.average()

# Plot EOG artifacts after ICA
plot_eog_artifacts(reconst_raw, eog_evoked_ica, 'After ICA Rejection')



Fitting ICA to data using 59 channels (please be patient, this may take a while)
Selecting by number: 25 components
Fitting ICA took 17.2s.
Using EOG channel: EOG 061
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 6007 samples (10.001 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
138 matching events found
No baseline correction applied
0 projection items activated
Applying ICA to Raw instance
    Transforming to ICA space (25 components)
    Zeroing out 1 ICA component
    Projecting back using 59 PCA components
Not setting metadata
145 matching events found
Setting baseline interval to [-0.19979521315838786, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 145 events and 421 original time points ...
0 bad epochs dropped
Using EOG channel: EOG 061
EOG channel index for this subject is: [68]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG 061 for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
-

In [7]:
##############################################
# Method 2: Signal Space Projection (SSP)
##############################################

# Compute SSP projections from EOG artifacts
projs_eog, events_eog = mne.preprocessing.compute_proj_eog(raw_ssp, n_grad=0, n_mag=0, n_eeg=3,
                                                          average=True)

# Apply SSP projections
raw_ssp.add_proj(projs_eog)
raw_ssp.apply_proj()

# Plot projectors
mne.viz.plot_projs_topomap(projs_eog, info=raw_ssp.info)

# Create epochs after SSP correction
epochs_ssp = Epochs(raw_ssp, events, event_id, tmin, tmax, proj=True,
                   picks=['eeg', 'eog'], baseline=(None, 0), preload=True)

# Create EOG epochs after SSP for visualization
eog_epochs_ssp = create_eog_epochs(raw_ssp, ch_name='EOG 061') 
eog_evoked_ssp = eog_epochs_ssp.average()

# Plot EOG artifacts after SSP
plot_eog_artifacts(raw_ssp, eog_evoked_ssp, 'After SSP Rejection')



Including 0 SSP projectors from raw file
Running EOG SSP computation
Using EOG channel: EOG 061
EOG channel index for this subject is: [68]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG 061 for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 6007 samples (10.001 s)

Now detecting blinks and generating corresponding events
Found 46 significant peaks
Number of EOG events detected: 46
Computing projector
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 35 Hz

FIR filter parameters
--

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 241 original time points ...
0 bad epochs dropped
No channels 'grad' found. Skipping.
No channels 'mag' found. Skipping.
Adding projection: eeg--0.200-0.200-PCA-01 (exp var=98.9%)
Adding projection: eeg--0.200-0.200-PCA-02 (exp var=0.9%)
Adding projection: eeg--0.200-0.200-PCA-03 (exp var=0.1%)
Done.
3 projection items deactivated
Created an SSP operator (subspace dimension = 3)
3 projection items activated
SSP projectors applied...
Not setting metadata
145 matching events found
Setting baseline interval to [-0.19979521315838786, 0.0] s
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 145 events and 421 original time points ...
0 bad epochs dropped
Using EOG channel: EOG 061
EOG channel index for this subject is: [68]
Filtering the

In [8]:

##############################################
# Compare the two methods: ICA vs SSP
##############################################

# Function to compare data before and after artifact rejection
def compare_methods(epochs_orig, epochs_ica, epochs_ssp, channel_idx=0):
    """Compare original data with ICA and SSP cleaned data."""
    plt.figure(figsize=(15, 10))
    
    # Original data
    plt.subplot(3, 1, 1)
    plt.plot(epochs_orig.times, epochs_orig.get_data()[:5, channel_idx, :].T)
    plt.title('Original EEG Data (5 epochs)')
    plt.xlabel('Time (s)')
    plt.ylabel('µV')
    
    # ICA cleaned data
    plt.subplot(3, 1, 2)
    plt.plot(epochs_ica.times, epochs_ica.get_data()[:5, channel_idx, :].T)
    plt.title('ICA Cleaned EEG Data (5 epochs)')
    plt.xlabel('Time (s)')
    plt.ylabel('µV')
    
    # SSP cleaned data
    plt.subplot(3, 1, 3)
    plt.plot(epochs_ssp.times, epochs_ssp.get_data()[:5, channel_idx, :].T)
    plt.title('SSP Cleaned EEG Data (5 epochs)')
    plt.xlabel('Time (s)')
    plt.ylabel('µV')
    
    plt.tight_layout()
    plt.show()

# Compare methods using a frontal channel (more affected by eye artifacts)
frontal_ch_idx = 0  # Choose a frontal channel index
for i, ch_name in enumerate(epochs.ch_names):
    if 'EOG' not in ch_name and ('Fp' in ch_name or 'F' in ch_name):
        frontal_ch_idx = i
        break

compare_methods(epochs, epochs_ica, epochs_ssp, channel_idx=frontal_ch_idx)

# Power spectrum comparison
def compare_psd(epochs_orig, epochs_ica, epochs_ssp):
    """Compare power spectral density before and after artifact rejection."""
    plt.figure(figsize=(12, 8))
    
    # Original data
    plt.subplot(3, 1, 1)
    epochs_orig.plot_psd(ax=plt.gca(), fmin=1, fmax=45, average=True, show=False)
    plt.title('PSD: Original Data')
    
    # ICA cleaned data
    plt.subplot(3, 1, 2)
    epochs_ica.plot_psd(ax=plt.gca(), fmin=1, fmax=45, average=True, show=False)
    plt.title('PSD: ICA Cleaned Data')
    
    # SSP cleaned data
    plt.subplot(3, 1, 3)
    epochs_ssp.plot_psd(ax=plt.gca(), fmin=1, fmax=45, average=True, show=False)
    plt.title('PSD: SSP Cleaned Data')
    
    plt.tight_layout()
    plt.show()

compare_psd(epochs, epochs_ica, epochs_ssp)

# Plot evoked responses to compare methods
evoked_orig = epochs.average()
evoked_ica = epochs_ica.average()
evoked_ssp = epochs_ssp.average()

plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
evoked_orig.plot(axes=plt.gca(), titles={'eeg': 'Original Evoked Response'}, show=False)

plt.subplot(3, 1, 2)
evoked_ica.plot(axes=plt.gca(), titles={'eeg': 'ICA Cleaned Evoked Response'}, show=False)

plt.subplot(3, 1, 3)
evoked_ssp.plot(axes=plt.gca(), titles={'eeg': 'SSP Cleaned Evoked Response'}, show=False)

plt.tight_layout()
plt.show()

# Create a summary table of the comparison
print("\nComparison of ICA vs SSP for Artifact Rejection:")
print("------------------------------------------------")
print("ICA:")
print(" - Pros: More specific removal of artifacts, preserves more signal")
print(" - Cons: More computation, requires manual selection of components")
print("SSP:")
print(" - Pros: Faster, simpler implementation")
print(" - Cons: May remove some brain signal along with artifacts")
print(" - Better for well-characterized artifact patterns")

# Example of topographic plot to compare
times = np.linspace(0.05, 0.15, 3)

# Create a figure for each method
fig_orig = evoked_orig.plot_topomap(times=times, colorbar=True, show=False)
plt.suptitle('Original: Topographic Maps', fontsize=16)

fig_ica = evoked_ica.plot_topomap(times=times, colorbar=True, show=False)
plt.suptitle('ICA Cleaned: Topographic Maps', fontsize=16)

fig_ssp = evoked_ssp.plot_topomap(times=times, colorbar=True, show=False)
plt.suptitle('SSP Cleaned: Topographic Maps', fontsize=16)

plt.show()

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
    Using multitaper spectrum estimation with 7 DPSS windows
Plotting power spectral density (dB=True).
Averaging across epochs before plotting...
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
    Using multitaper spectrum estimation with 7 DPSS windows
Plotting power spectral density (dB=True).
Averaging across epochs before plotting...
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
    Using multitaper spectrum estimation with 7 DPSS windows
Plotting power spectral density (dB=True).
Averaging across epochs before plotting...


/tmp/ipykernel_204855/2267245937.py:83: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()



Comparison of ICA vs SSP for Artifact Rejection:
------------------------------------------------
ICA:
 - Pros: More specific removal of artifacts, preserves more signal
 - Cons: More computation, requires manual selection of components
SSP:
 - Pros: Faster, simpler implementation
 - Cons: May remove some brain signal along with artifacts
 - Better for well-characterized artifact patterns
